In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
new_data = fetch_20newsgroups(subset='all',random_state=156)

In [8]:
news_data = new_data

In [9]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [5]:
import pandas as pd

In [10]:
print("target 클래스 값과 분포도 : ",pd.Series(news_data.target).value_counts().sort_index())
print("target 클래스의 이름들 : ",news_data.target_names)

target 클래스 값과 분포도 :  0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 :  ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [13]:
print(news_data['data'][0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [17]:
# subset='train'으로 학습용 데이터만 추출
# remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'로 테스트 데이터만 추출, remove=('headers','footers','qutes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

print(len(train_news.data))
print(len(test_news.data))

11314
7532


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수해
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape : ',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape :  (11314, 101631)


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)

pred = lr_clf.predict(X_test_cnt_vect)
print("CountVectorized Logistic Regression의 예측 정확도 : ",accuracy_score(y_test,pred))

CountVectorized Logistic Regression의 예측 정확도 :  0.6072756240042485


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## TF-IDF

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF LogisticRegression의 예측 정확도 : ",accuracy_score(y_test,pred))

TF-IDF LogisticRegression의 예측 정확도 :  0.6736590546999469


In [32]:
# 파라미터 튜닝

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)

pred = lr_clf.predict(X_test_tfidf_vect)
print("TF-IDF vectorized LogisticRegression의 예측 정확도 : ",accuracy_score(y_test,pred))

TF-IDF vectorized LogisticRegression의 예측 정확도 :  0.6922464152947424


In [34]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. cv는 폴드 세트로 설정
params = {"C":[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print("LogisticRegression best C parameter : ",grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 평가
pred =grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized LogisticRegression의 예측 정확도 : ",accuracy_score(y_test,pred))

## Pipe Line

In [38]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
                    ('lr_clf',LogisticRegression(random_state=156))])

pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)

print("PipleLine 통한 LogisticRegression 예측 정확도 : ",accuracy_score(y_test,pred))

PipleLine 통한 LogisticRegression 예측 정확도 :  0.6922464152947424
